# Groth16

## Introduction
Groth16 is a zk-SNARKs algorithm that allow a prover to prove that a statement is true, without revealing any other information about the input.

In this jupyter notebook, we are going to implement Groth16 to prove that our statement to a problem is true.

## Context and Problem

In the following sections, we will implement and use Groth16 to prove a statement without leaking any other informations.

The problem that we are going to prove is: "I know $\{x,y,z\}$ such that $x^3+3x^2 y + 2z^2y^3 = 169695$".

## Problem's solution
The prover needs to know the values ${x,y,z}$.
In our example, those values are:
$$x = 3$$
$$y = 12$$
$$z = 7$$
Those values will remain private, they will never be sent to the verifier.

## Initialize Python setup
In our implementation, we will be working on the BN128 elliptic curve.
As such, we will use a finite field which is the number of point of this curve (the order of the curve).
We will need to adapt the our different computing to this.

In Python, we will work with the `galois` library for the finite field.
For the EC implementation, we will use `py_ecc`. `numpy` will be used to manipulate matrices.

In [9]:
from py_ecc.bn128 import G1, multiply, add, curve_order, eq, Z1
import galois

print("Initializing a large field...")
GF = galois.GF(curve_order)
print("Field initialized")

Initializing a large field...
Field initialized


## Arithmetic Circuit
Our equation $out = x^3+3x^2 y + 2z^2y^3$ can be broken down into an arithmetic circuit.
The following scheme shows one of the multiple ways to represent our equation as an arithmetic circuit.

![Equation as an arithmetic circuit](./images/arithmetic-circuit.png)

From this arithmetic circuit, we can define a set of constraints which has the form $O = L * R$. *Note: in the following set of constraints, $v1, ..., v7$ represent intermediate values.*
$$v_1 = x * x$$<!-- x^2 -->
$$v_2 = v_1 * x$$<!-- x^3 -->
$$v_3 = 3v_1 * y$$<!-- 3x^2 y-->
$$v_4 = y * y$$<!-- y^2 -->
$$v_5 = v_4 * y$$<!-- y^3 -->
$$v_6 = 2z * z$$<!-- 2z^2-->
$$out = v_2 + v_3 + v_5 * v_6$$ <!-- x^3 + 3x^2 y + 2z^2 y^3-->

## Rank 1 Constraint System

Now that we have our arithmetic circuit, we can turn it into a Rank 1 Constraint System.
We need to adapt our set of constraints to easily match the required R1CS format:
$$v_1 = x * x$$<!-- x^2 -->
$$v_2 = v_1 * x$$<!-- x^3 -->
$$v_3 = 3v_1 * y$$<!-- 3x^2 y-->
$$v_4 = y * y$$<!-- y^2 -->
$$v_5 = v_4 * y$$<!-- y^3 -->
$$v_6 = 2z * z$$<!-- 2z^2-->
$$out - v_2 - v_3 = v_5 * v_6$$ <!-- x^3 + 3x^2 y + 2z^2 y^3-->

Now that we have a proper $O = L * R$ form, we can create 4 matrices.
The first matrix is named $s$. It is known as the *witness* matrix.
The value of $s$ elements are known by the prover.
$$s = \begin{bmatrix} 1 & out & x & y & z & v_1 & v_2 & v_3 & v_4 & v_5 & v_6 \end{bmatrix}$$
$$s = \begin{bmatrix} 1 & 169695 & 3 & 12 & 7 & 9 & 27 & 324 & 144 & 1728 & 98 \end{bmatrix}$$

Then, we can produce 3 matrices that will represent the constraints.
It sort of represents the way the $s$ elements are interconnected.
Those 3 matrices will be named $L$, $R$ and $O$.
$$L = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 3 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
\end{bmatrix}$$
$$R = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}$$
$$O = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 0 & 0 & -1 & -1 & 0 & 0 & 0 \\
\end{bmatrix}$$

The final R1CS representation is $Ls \odot Rs = Os$.
Each column of the matrices $L,R,O$ corresponds to an element of $s$.
Each row of the matrices $L,R,O$ corresponds to one of the constraints of the arithmetic circuit.

For example, let's take the 3rd row of $L, R, O$.
In $L$, the value $3$ is set at the 6th column.
The 6th column of $s$ is the $v_1$ variable.
So, the 3rd row of $L$ encodes $3 v_1$.
The same logic applies to $R$, its 3rd row encodes $y$.
And for $O$, the 3rd row encodes $v_3$.
If we put it all together, the $Os = Ls \odot Rs$ gives $v_3 = v_1 * y$ which correspond to our 3rd constraints defined previously.

Let's verify that the computing of $Os = Ls \odot Rs$ is correct.
We will use Python and numpy for this.
As we are working on a finite field, negative number will be modified.
For example, in $F_p$, $-1 (mod\ p)= p-1$.

In [12]:
import numpy as np

# define the matrices in our Galois Field

L = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]))
R = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]))


O = GF(np.array([
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, curve_order-1, curve_order-1, 0, 0, 0]]))

# witness vector `s`
s = GF(np.array([1, 169695, 3, 12, 7, 9, 27, 324, 144, 1728, 98]))

# Calculate the result of the mutliplication of matrix, and check if it is equal to O*s
result = np.matmul(O, s) == np.matmul(L, s) * np.matmul(R, s)

# check that every element-wise equality is true
print("Os = Ls * Rs:", result.all())

Os = Ls * Rs: [ True  True  True  True  True  True  True]


## QAP

We have a working R1CS corresponding to our arithmetic circuit.
The next step is to transform this R1CS into a Quadratic Arithmetic Program (QAP).

This transformation consists of turning our constraint matrices into polynomials.
The main goal of this transformation is **succinctness**.
Thanks to the Schwartz-Zippel Lemma, we know that evaluating two polynomials at a random point $x$ allows to nearly be certain that they are the same polynomial if they return the same value $y$.
In summary, if $P_1(\tau) == P_2(\tau)$, then $P1 == P2$, with $\tau$ being a random number.
This evaluation is way more succinct than matrices evaluation.

To achieve succinctness, we will derivate a QAP from our R1CS.
Our goal is to get $u * v = w$ if $Ls \odot Rs = Os$.
The transformation is made possible thanks to a ring homomorphism between matrices and polynomials.

The created polynomials must encode the same informations as our R1CS matrices, but uniquely at the points at which it will be evaluated.
As we have 7 constraints, we will evaluate our polynomials at $x=[1,2,3,4,5,6,7]$.

### R1CS to QAP
For each column of each matrix, we will interpolate a polynomial.
The interpolation will be done using Lagrange interpolation.
The interpolated polynomial will encode all the values of the column.

For example, the third column of $L$ is:
$$\begin{bmatrix} 1 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{bmatrix}$$

So, the interpolated polynomial $P_{L_3}$ must return the following values:
$$P_{L_3}(1) = 1$$
$$P_{L_3}(2) = 1$$
$$P_{L_3}(3) = 0$$
$$P_{L_3}(4) = 0$$
$$P_{L_3}(5) = 0$$
$$P_{L_3}(6) = 0$$
$$P_{L_3}(7) = 0$$

The following Python code shows the exact previous example (3rd column of matrix $L$).

In [16]:
x = GF(np.array([1,2,3,4,5,6,7]))
L3_column = GF(np.array([1,1,0,0,0,0,0]))

L3_poly = galois.lagrange_poly(x, L3_column)

print("The resulting polynomial is:\n", L3_poly, sep='')

# Checking each column element
print("L3_poly(1) == 1:", L3_poly(1) == 1)
print("L3_poly(2) == 1:", L3_poly(2) == 1)
print("L3_poly(3) == 0:", L3_poly(3) == 0)
print("L3_poly(4) == 0:", L3_poly(4) == 0)
print("L3_poly(5) == 0:", L3_poly(5) == 0)
print("L3_poly(6) == 0:", L3_poly(6) == 0)
print("L3_poly(7) == 0:", L3_poly(7) == 0)

The resulting polynomial is:
152001686609994966821155595453175521448252530558444682942348640184554225664x^6 + 9211302208565694989362029084462436599764103351841747786306327595183986075239x^5 + 18392204079809390985359827049834238095238556197571806636024185462331061305343x^4 + 10488116376089652710659736086269110979929424608532683123022056172734241570826x^3 + 14288158541339526881188625972598499016135737872493800196580772177348097212391x^2 + 13132945723103565133347843447154365053129018640249620606218922511945485097403x + 21888242871839275222246405745257275088548364400416034343698204186575808495603
L3_poly(1) == 1: True
L3_poly(2) == 1: True
L3_poly(3) == 0: True
L3_poly(4) == 0: True
L3_poly(5) == 0: True
L3_poly(6) == 0: True
L3_poly(7) == 0: True


Now that we achieved it for one column,
we can automate the process to get the interpolated polynomials for all the columns of each matrix.

The following Python code does it!

In [56]:
def interpolate_column(col, nb):
    xs = GF(np.arange(1,nb+1))
    return galois.lagrange_poly(xs, col)

def get_polys_of_matrix(matrix):
    polys = []
    nb_of_rows = len(matrix)
    nb_of_columns = len(matrix[0])
    # for each column
    for col_id in range(nb_of_columns):
        column = []
        for row in range(nb_of_rows):
            column.append(matrix[row][col_id])
        polys.append(interpolate_column(GF(np.array(column)), nb_of_rows))
    return np.array(polys)
    

## computes all interpolated polynomials for L, R, and O
U_polys = get_polys_of_matrix(L)
V_polys = get_polys_of_matrix(R)
W_polys = get_polys_of_matrix(O)

### Adding the witness to the polynomials

Once the interpolated polynomials $U,V,W$ have been calculated for $L,R,O$, we can plug the witness to them.

This step is pretty simple. As one column represents one witness element and one column is interpolated as one polynomial,
then multiplying the witness element (a scalar) by the polynomial is enough.
Let's get our final polynomials $U_a, V_a, W_a$!

In [63]:
def add_witness_to_polys(polys, witness):
    ret_polys = []
    for i in range(len(polys)):
        ret_polys.append(polys[i] * witness[i])
    return np.array(ret_polys)

## plug witness into the three polynomials collections
U_a = add_witness_to_polys(U_polys, s)
V_a = add_witness_to_polys(V_polys, s)
W_a = add_witness_to_polys(W_polys, s)

### Verifying the QAP

Now that we have our $U_a, V_a, W_a$ polynomials, we can check to see if the $U_a * V_a = W_a$ form herited from our R1CS still holds.

In fact, the real equation of a QAP is $\sum\limits_{i=0}^{m} a_i u_i(x) \sum\limits_{i=0}^{m} a_i v_i(x) = \sum\limits_{i=0}^{m} a_i w_i(x)$.

The following code verifies the QAP equation for all our polynomials.
*Note: For the 7th constraint, remember that the $O$ part of the R1CS was $out-v2-v3$.*

In [98]:
# Summing all the polynamials of the collection into one polynomial
Ua = galois.Poly([0], field=GF)
Va = galois.Poly([0], field=GF)
Wa = galois.Poly([0], field=GF)
for i in range(len(U_a)):
    Ua += U_a[i]
    Va += V_a[i]
    Wa += W_a[i]

for j in range(1, len(L)+1):
    print('[{0}]th constraints holds? {1}, value is {2}'.format(str(j), Ua(j) * Va(j) == Wa(j), Wa(j)))

[1]th constraints holds? True, value is 9
[2]th constraints holds? True, value is 27
[3]th constraints holds? True, value is 324
[4]th constraints holds? True, value is 144
[5]th constraints holds? True, value is 1728
[6]th constraints holds? True, value is 98
[7]th constraints holds? True, value is 169344


### QAP balancing
Our QAP $\sum\limits_{i=0}^{m} a_i u_i(x) \sum\limits_{i=0}^{m} a_i v_i(x) = \sum\limits_{i=0}^{m} a_i w_i(x)$ has an issue.
The left term will have a degree greater than the right term.
Even if this does not have any impact on the points at which we evaluate those polynomials, we will fix it by adding a *balancing term*.

The balancing term is a set of two polynomial $h(x)$ and $t(x)$ that will be added to the right term:
$\sum\limits_{i=0}^{m} a_i u_i(x) \sum\limits_{i=0}^{m} a_i v_i(x) = \sum\limits_{i=0}^{m} a_i w_i(x) + h(x)t(x)$.

We need this balancing term to be zero at the points we evaluate the polynomials. To do so, we use $t(x)$ as a sort of nullifier.
As we evaluate at $x=[1,...,7]$, we can easily find that $t(x)= (x-1)(x-2)...(x-7)$. Remember that as we are in a finite field, negative values are handled differently.

Then, we need to compute $h(x)$. It is easy to do so as we know all the terms:
$$ \frac{(U \cdot a)(V \cdot a) - (W \cdot a)}{t(x)} = h(x)$$

In [100]:
# Computing t(x)
## T = (x-1)
T = galois.Poly([1, curve_order-1], field=GF)
## Then multiply by the other values: (x-2)(x-3)...(x-7)
for i in range(2, len(L)+1):
    T = T * galois.Poly([1, curve_order-i], field=GF)

H = (Ua + Va - Wa) // T
print("t(x):", T)
print("h(x):", H)

t(x): x^7 + 21888242871839275222246405745257275088548364400416034343698204186575808495589x^6 + 322x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808493657x^4 + 6769x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808482485x^2 + 13068x + 21888242871839275222246405745257275088548364400416034343698204186575808490577
h(x): 0


In [101]:
# Let's verify that the equation still holds when we add h(x)t(x)
for j in range(1, len(L)+1):
    print('[{0}]th constraints holds? {1}, value is {2}'.format(str(j), Ua(j) * Va(j) == Wa(j) + H(j)*T(j), Wa(j)))

[1]th constraints holds? True, value is 9
[2]th constraints holds? True, value is 27
[3]th constraints holds? True, value is 324
[4]th constraints holds? True, value is 144
[5]th constraints holds? True, value is 1728
[6]th constraints holds? True, value is 98
[7]th constraints holds? True, value is 169344
